In [144]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2023-ml-w4p2/train.csv
/kaggle/input/2023-ml-w4p2/test.csv
/kaggle/input/2023-ml-w4p2/sample_submit.csv


In [145]:
# 데이터 로드

train_df = pd.read_csv('/kaggle/input/2023-ml-w4p2/train.csv')
print(train_df)
test_df = pd.read_csv('/kaggle/input/2023-ml-w4p2/test.csv')
print(test_df)
submit = pd.read_csv('/kaggle/input/2023-ml-w4p2/sample_submit.csv')
print(submit)


          ID  company      model  year  transmission  mileage  fueltype  tax  \
0          0        4      Auris  2015             1    47541         4  145   
1          1        0         X2  2018             3     5000         0  145   
2          2        2      Focus  2017             3    47018         4  145   
3          3        2      Focus  2020             1     1550         4  145   
4          4        0   3 Series  2017             3    23505         0  145   
...      ...      ...        ...   ...           ...      ...       ...  ...   
34165  34165        0   1 Series  2017             1    39201         4  150   
34166  34166        2     Fiesta  2019             1     2000         4  145   
34167  34167        2     Fiesta  2019             1    15820         4  145   
34168  34168        4      Hilux  2018             1    13099         0  260   
34169  34169        2     Fiesta  2013             1    32638         4   30   

        mpg  engineSize  price  
0     

In [146]:
# x 데이터 설정, ID, price 제외 나머지
x_data = train_df.drop(['ID', 'price'], axis = 1)
print(x_data)

# test 데이터에도 ID가 들어가있으므로 제외

test_df = test_df.drop('ID', axis = 1)
print(test_df)


       company      model  year  transmission  mileage  fueltype  tax   mpg  \
0            4      Auris  2015             1    47541         4  145  46.3   
1            0         X2  2018             3     5000         0  145  50.4   
2            2      Focus  2017             3    47018         4  145  51.4   
3            2      Focus  2020             1     1550         4  145  49.6   
4            0   3 Series  2017             3    23505         0  145  64.2   
...        ...        ...   ...           ...      ...       ...  ...   ...   
34165        0   1 Series  2017             1    39201         4  150  36.2   
34166        2     Fiesta  2019             1     2000         4  145  58.9   
34167        2     Fiesta  2019             1    15820         4  145  60.1   
34168        4      Hilux  2018             1    13099         0  260  40.4   
34169        2     Fiesta  2013             1    32638         4   30  54.3   

       engineSize  
0             1.6  
1          

In [147]:
# y 데이터 설정
y_data = train_df[['price']]
print(y_data)


       price
0       8095
1      21726
2      10490
3      17490
4      18995
...      ...
34165  18480
34166  16495
34167  13000
34168  20998
34169   6800

[34170 rows x 1 columns]


In [148]:
# 카테고리 실수화, model 부분 실수화
from sklearn.preprocessing import LabelEncoder #LabelEncoder method, 리발에 실수가 아닌부분 : model
# fit_transform을 사용하지 않는이유?
#학습데이터 세트에서 변환을 위한 기반 설정(예를 들어 학습 데이터 세트의 최대값/최소값등)을 먼저 fit()을 통해서 설정한 뒤에 이를 기반으로 학습 데이터의 transform()을 수행하되 학습 데이터에서 설정된 변환을 위한 기반 설정을 그대로 테스트 데이터에도 적용하기 위해서입니다.
#이부분 질문필요

classle = LabelEncoder()
classle.fit(x_data['model'].values)

train_LE = classle.transform(x_data['model'].values) # fit-transform
test_LE = classle.transform(test_df['model'].values) # fit-transform

# 실수화된 부분(model) 부분 덮어쓰기
x_data['model'] = train_LE
test_df['model'] = test_LE

print(x_data)
print(test_df)

       company  model  year  transmission  mileage  fueltype  tax   mpg  \
0            4     15  2015             1    47541         4  145  46.3   
1            0     87  2018             3     5000         0  145  50.4   
2            2     26  2017             3    47018         4  145  51.4   
3            2     26  2020             1     1550         4  145  49.6   
4            0      2  2017             3    23505         0  145  64.2   
...        ...    ...   ...           ...      ...       ...  ...   ...   
34165        0      0  2017             1    39201         4  150  36.2   
34166        2     25  2019             1     2000         4  145  58.9   
34167        2     25  2019             1    15820         4  145  60.1   
34168        4     33  2018             1    13099         0  260  40.4   
34169        2     25  2013             1    32638         4   30  54.3   

       engineSize  
0             1.6  
1             2.0  
2             1.0  
3             1.0  

In [154]:
# KNN regression 적용, 좋다 vs 나쁘다 레이블 분류가 아니라 가격을 예측하는것이기 때문에 회귀 사용, Classifier 대신
from sklearn.neighbors import KNeighborsRegressor

regressor = KNeighborsRegressor()

regressor.fit(x_data, y_data) # x와 y데이터에 대한 학습

regressor.predict(x_data)
test_PD = regressor.predict(x_data)
len(test_PD)

from sklearn import metrics


test_PD = regressor.predict(test_df) # test파일을 교육 후 저장
print(test_PD)
print(regressor.score(regressor.score(test_df, test_PD)))

[[13583.32791594]
 [ 9703.06106228]
 [15164.43442871]
 ...
 [13701.79678236]
 [22987.48600956]
 [ 8240.55635893]]


TypeError: score() missing 1 required positional argument: 'y'

In [150]:
# 입력 데이터 표준화 
from sklearn.preprocessing import StandardScaler

std=StandardScaler()
std.fit(x_data) # 표준화
X_train_std=std.fit_transform(x_data) # normalized training data
X_test_std=std.transform(test_df) # nomarlized test data

print(x_data)
X_train_std[1:5, ]

       company  model  year  transmission  mileage  fueltype  tax   mpg  \
0            4     15  2015             1    47541         4  145  46.3   
1            0     87  2018             3     5000         0  145  50.4   
2            2     26  2017             3    47018         4  145  51.4   
3            2     26  2020             1     1550         4  145  49.6   
4            0      2  2017             3    23505         0  145  64.2   
...        ...    ...   ...           ...      ...       ...  ...   ...   
34165        0      0  2017             1    39201         4  150  36.2   
34166        2     25  2019             1     2000         4  145  58.9   
34167        2     25  2019             1    15820         4  145  60.1   
34168        4     33  2018             1    13099         0  260  40.4   
34169        2     25  2013             1    32638         4   30  54.3   

       engineSize  
0             1.6  
1             2.0  
2             1.0  
3             1.0  

array([[-1.51026792,  1.96991151,  0.47662902,  1.83851219, -0.87994672,
        -1.18757679,  0.40725992, -0.32237328,  0.54882437],
       [ 0.03447068, -0.26065734,  0.01252285,  1.83851219,  1.0740513 ,
         0.89248979,  0.40725992, -0.26845529, -1.12125567],
       [ 0.03447068, -0.26065734,  1.40484136, -0.18376245, -1.04038494,
         0.89248979,  0.40725992, -0.36550768, -1.12125567],
       [-1.51026792, -1.1382582 ,  0.01252285,  1.83851219, -0.0193933 ,
        -1.18757679,  0.40725992,  0.42169506,  0.54882437]])

In [151]:
# 입력데이터 똑같이 회귀시킴
from sklearn.neighbors import KNeighborsRegressor

regressor = KNeighborsRegressor(n_neighbors = 3, weights = 'distance')

regressor.fit(X_train_std, y_data) # 표준화 시킨 데이터에 관한 학습

test_PD_std = regressor.predict(X_test_std) # 예측
print(test_PD_std)


[[ 8373.27981686]
 [ 9236.86396108]
 [20035.26017867]
 ...
 [14107.81380575]
 [29937.00881982]
 [11422.80236832]]


ValueError: X has 1 features, but KNeighborsRegressor is expecting 9 features as input.

In [ ]:
submit['price'] = test_PD_std
submit = submit.astype(np.int64)
submit.to_csv('submit.csv', mode='w', header=True, index=False)